In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn import metrics
import itertools
from sklearn.linear_model import PassiveAggressiveClassifier
import re
import time
import pickle
import zlib

import nltk 
from nltk.tokenize import word_tokenize 
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

In [2]:
df = pd.read_csv('data.csv')
df=df.dropna()
df.loc[df['Label']== 0, 'Label'] = 'REAL'
df.loc[df['Label']== 1, 'Label'] = 'FAKE'
df.shape
df

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,FAKE
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",FAKE
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,FAKE
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,FAKE
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",FAKE
...,...,...,...,...
4003,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",REAL
4004,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,REAL
4005,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,REAL
4007,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,FAKE


In [15]:
df['text']=df['Headline']+' '+df['Body']

In [16]:
preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(df['text'].values):
    #sent = decontracted(sentance)
    sentance = sentance.replace('\\r', ' ')
    sentance = sentance.replace('\\"', ' ')
    sentance = sentance.replace('\\n', ' ')
    sentance = re.sub('[^A-Za-z0-9]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    #sentance = ' '.join(e for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_text.append(sentance.lower().strip())

df[''] = preprocessed_text

100%|████████████████████████████████████████████████████████████████████████████| 3988/3988 [00:00<00:00, 4185.16it/s]


In [17]:
y = df['Label']
X = df['text']

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,)

In [19]:
tfidf_vect = TfidfVectorizer(max_features=5000)
tfidf_train = tfidf_vect.fit_transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vect.get_feature_names())

In [20]:
filepath = 'web\\vectorizer.pkl'
pickle.dump(tfidf_vect, open(filepath, 'wb'))

In [21]:
clf = MultinomialNB() 
clf.fit(tfidf_train, y_train)                      
pred = clf.predict(tfidf_test)                     
score = metrics.accuracy_score(y_test, pred)
print("The Score is: ")  
print(score)
cm = metrics.confusion_matrix(y_test, pred)
print(cm)

The Score is: 
0.9365079365079365
[[551  18]
 [ 58 570]]


In [22]:
linear_clf = PassiveAggressiveClassifier()
linear_clf.fit(tfidf_train, y_train)
pred = linear_clf.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
print("The Score is: ")  
print(score)
cm = metrics.confusion_matrix(y_test, pred)
print(cm)

The Score is: 
0.9874686716791979
[[559  10]
 [  5 623]]


In [23]:
clf = GradientBoostingClassifier()
clf.fit(tfidf_train, y_train)
pred = clf.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
print("The Score is: ")  
print(score)
cm = metrics.confusion_matrix(y_test, pred)
print(cm)

The Score is: 
0.9841269841269841
[[562   7]
 [ 12 616]]


In [24]:
clf1 = LogisticRegression(C=9)
clf1.fit(tfidf_train,y_train)
sc2 = clf1.score(tfidf_test,y_test)
print("The Score is: ")    
print(sc2)

The Score is: 
0.9782790309106099


In [25]:
clf1  = RandomForestClassifier(n_estimators=100, random_state=1, n_jobs=-1, class_weight='balanced')
clf1.fit(tfidf_train, y_train)
clf1.score(tfidf_test, y_test)

0.9799498746867168

In [26]:
clf = SVC(C=100, gamma=0.1,probability=True)
clf.fit(tfidf_train, y_train)
pred = clf.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
print("The Score is: ")  
print(score)
cm = metrics.confusion_matrix(y_test, pred)
print(cm)

The Score is: 
0.9899749373433584
[[561   8]
 [  4 624]]


In [27]:
filepath = 'web\\classifier.pkl'
pickle.dump(clf1, open(filepath, 'wb'))